# Assignment 3 - 2C: Implementing a Large Deformation with Point Load to Capture Failure

## Imports and Setup

In [ ]:
%matplotlib inline
from finiteelementanalysis import pre_process as pre
from finiteelementanalysis import pre_process_demo_helper_fcns as pre_demo
from finiteelementanalysis.solver import hyperelastic_solver
from finiteelementanalysis import visualize as viz
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# for saving files later
tutorials_dir = Path.cwd()  # Use current working directory in notebook

## Beam Geometry and Base Parameters

In [ ]:
L = 30   # length in x
H = 1    # height in y
P = -1000  # point load (large downward force to induce failure)
E = 1000.0
nu = 0.3
mu = E / (2.0 * (1.0 + nu))
kappa = E / (3.0 * (1.0 - 2.0 * nu))
material_props = np.array([mu, kappa])

# Analytical solution for comparison (point load at end)
E_eff = E / (1 - nu ** 2.0)  # Plane strain adjustment
I = H ** 3 / 12.0
w_analytical = P * L ** 3 / (3.0 * E_eff * I)

## Simulation Function

In [ ]:
# Function to run simulation and return tip deflection
def run_simulation(ele_type, nx, ny, label):
    ndof = 2  # 2 DOFs per node (x, y)
    
    # Generate mesh
    coords, connect = pre.generate_rect_mesh_2d(ele_type, 0.0, 0.0, L, H, nx, ny)
    
    # Identify boundaries
    boundary_nodes, boundary_edges = pre.identify_rect_boundaries(
        coords, connect, ele_type, x_lower=0.0, x_upper=L, y_lower=0.0, y_upper=H
    )
    
    # Boundary conditions
    fixed_left = pre.assign_fixed_nodes_rect(boundary_nodes, "left", 0.0, 0.0)
    
    # Apply point load at the tip node (x=L, y=H/2)
    tip_node = None
    tol = 1e-3
    for i, (x, y) in enumerate(coords):
        if abs(x - L) < tol and abs(y - H/2) < H/(2*ny):
            tip_node = i
            break
    if tip_node is None:
        raise ValueError(f"Could not find tip node near x=L, y=H/2 for {label}.")
    
    # Create dload_info for point load: [node_id, dof, load_value]
    dload_info = np.array([[tip_node, 1, P]])  # Apply P in y-direction (dof=1)
    
    fixed_nodes = fixed_left
    
    # Solve
    try:
        displacements_all, nr_info_all = hyperelastic_solver(
            material_props,
            ele_type,
            coords.T,
            connect.T,
            fixed_nodes,
            dload_info,
            nr_print=True,  # Enable printing to capture solver failure
            nr_num_steps=10,  # Increase steps to allow load incrementing
            nr_tol=1e-10,
            nr_maxit=30,
        )
    except Exception as e:
        error_msg = (
            f"Simulation failed for {label}:\n"
            f"  Exception Type: {type(e).__name__}\n"
            f"  Error Message: {str(e)}\n"
            f"  Load Step Info: Check solver output above (nr_print=True) for details on the last attempted step.\n"
            f"  Possible Cause: Large point load (P={P}) may cause excessive deformation or non-convergence in the Newton-Raphson solver."
        )
        print(error_msg)
        return None
    
    final_disp = displacements_all[-1]
    
    tip_disp_y = final_disp[ndof*tip_node + 1]
    
    # Generate deformation GIF
    img_name = f"deformation_{label}.gif"
    fname = str(tutorials_dir / img_name)
    viz.make_deformation_gif(displacements_all, coords, connect, ele_type, fname)
    
    return tip_disp_y

## Run Simulation

In [ ]:
# Define single case (no refinement)
case = {"label": "Original", "ele_type": "D2_nn4_quad", "nx": 40, "ny": 2}

# Run simulation
tip_deflection = run_simulation(case["ele_type"], case["nx"], case["ny"], case["label"])

## Print Results

In [ ]:
print("\n=== Tip Deflection Results ===")
print(f"Analytical Euler-Bernoulli deflection: {w_analytical:.6f}")
if tip_deflection is not None:
    error = abs(tip_deflection - w_analytical)
    print(f"{case['label']}:")
    print(f"  Computed tip deflection (y): {tip_deflection:.6f}")
    print(f"  Absolute error: {error:.6e}")
else:
    print(f"{case['label']}: Failed to converge")

## Plot Results

In [ ]:
# Prepare data for plotting
labels = [case['label']]
computed_deflection = [tip_deflection if tip_deflection is not None else 0]
analytical_line = [w_analytical]

# Create bar plot
plt.figure(figsize=(6, 6))
plt.bar(labels, computed_deflection, color='skyblue', label='Computed')
plt.plot(labels, analytical_line, color='red', linestyle='--', label='Analytical')
plt.xlabel('Simulation Case')
plt.ylabel('Tip Deflection (y)')
plt.title('Tip Deflection Result')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

## Additional Variable

In [ ]:
aa = 44